# Cyclic ID Algorithm: High-Level Workflow and Examples

This notebook will demonstrate the complete cyclic ID algorithm for identifying causal effects in cyclic graphs with observational data.


**This notebook will show:**

- Examples using a graph with Identifiable queries ✅
- Examples using a graph with Unidentifiable queries ❌

**Note**: This is a high-level demo showing the full workflow. For more information please see other implementation notebooks and explanation notebooks. 

## Setup

In [ ]:
from y0.dsl import Variable
from y0.graph import NxMixedGraph

# Define variables
X = Variable("X")
Y = Variable("Y")

## Example 1: Identifiable Cyclic Graph

### Graph Structure 

For this high level example we have a minimal example of **unidentifiability** first in a cyclic graph. 

- **X → X**: Self-loop on X (feedback)
- **X → Y**: X influences Y
- **Y → X**: Y influences X

X and Y form a feedback loop, which is also known as a strongly connected component.


### Graph Description

This graph has:

- **R → X**: External input to the cycle
- **X ⇄ Y**: Bidirectional cycle between X and Y
- **X → X**: Self-loop on X
- **X → Z**: Path from cycle to outcome

**Expected Result**: This query is expected to be identifiable. With a real graph, we usually would not know if it is identifiable or not. Various queries might either be identifiable or unidentifiable. 

In [ ]:
# Define Graph 1: Identifiable Cyclic Graph Example with an identifiable query.

graph_1 = NxMixedGraph.from_edges(
    directed=[
        (X, X),
        (X, Y),
        (Y, X),
    ]
)

print("Graph 1 created.")
print(f"Nodes: {list(graph_1.nodes())}")
print(f"Directed edges: {list(graph_1.directed.edges())}")

## Implementing the Cyclic ID Algorithm

Below we will implement the main Cyclic ID algorithm that will determine if the example graphs we have are identifiable or not.

**What the algorithm does:**

The Cyclic ID algorithm solves a fundamental problem: *What causal queries can be identified on a cyclic graph?*

**High-level strategy:**

1. **Find relevant variables**: Identify which variables are actually needed (ancestors of our outcome).
2. **Break into pieces**: Split the graph into manageable "districts", or groups of variables that need to be handled together.
3. **Identify each piece**: Use the IDCD algorithm to identify each district separately.
4. **Combine results**: Multiply the identified pieces together and focus on our outcome variable.

**Why this works for cycles:**

Traditional methods assume acyclic graphs (no feedback loops). This algorithm handles cycles by:
- Treating feedback loops as unified blocks (consolidated districts)
- Using a special ordering (apt-order) instead of topological order
- Recursively breaking down complex structures until each piece is identifiable

In [ ]:
# importing the necessary utility functions for the algorithm

from y0.algorithm.identify import Unidentifiable
from y0.algorithm.identify.idcd import idcd
from y0.algorithm.ioscm.utils import (
    get_apt_order,
    get_consolidated_district,
    get_graph_consolidated_districts,
    get_strongly_connected_components,
)
from y0.dsl import Probability, Product

### Running Cyclic ID Algorithm on Graph 1

Now below we can call the function on the first example graph to identify the query we have. We can do this same action for several queries in a directed mixed graph. First we see the use of a helper function in order to get the initial distribution.


In [ ]:
def initialize_district_distribution(graph, district, apt_order):
    """
    Initializes the probability distribution for a given district before identification. # noqa: D401

    This implements Proposition 9.8(3): each district's initial distribution is built
    by finding its strongly connected components (feedback loops), computing each
    component's distribution conditioned on variables that come before it, then
    multiplying these distributions together.

    param graph: Causal graph
    param district: Set of variables representing the district
    param apt_order: Ordering of all variables that respsects the causal structure.
    """
    # find all the feedback loops within the district
    district_subgraph = graph.subgraph(district)
    feedback_loops = get_strongly_connected_components(district_subgraph)  # feedback loops = SCCs

    loop_distributions = []

    for loop in feedback_loops:
        # find where the loop appears in the variable ordering
        # use the earliest position if the loop has multiple variables
        loop_positions = [apt_order.index(v) for v in loop if v in apt_order]

        if loop_positions:
            earliest_position = min(loop_positions)
            # get all variables that come before this loop in the ordering
            variables_before = set(apt_order[:earliest_position])
        else:
            variables_before = set()

        # compute the probability distribution for this loop
        if variables_before:
            # we do have predecessors - create P(loop | predecessors)
            all_vars = set(loop) | variables_before
            loop_distribution = Probability.safe(all_vars).conditional(
                variables_before
            )  # or condition on the predecessors
        else:
            # no predecessors - create P(loop)
            loop_distribution = Probability.safe(set(loop))

        loop_distributions.append(loop_distribution)

    # multiply all the loop distributions together
    if len(loop_distributions) == 1:
        return loop_distributions[0]
    else:
        return Product.safe(loop_distributions)

In [ ]:
def cyclic_id(graph, outcomes, interventions): # noqa: D103
    # line 2 - validate preconditions
    # require: Y ⊆ V, W ⊆ V, Y ∩ W = ∅

    all_nodes = set(graph.nodes())

    if not outcomes.issubset(all_nodes):
        raise ValueError("Outcomes must be a subset of the graph's nodes.")

    if not interventions.issubset(all_nodes):
        raise ValueError("Interventions must be a subset of the graph's nodes.")

    if outcomes & interventions:
        raise ValueError("Outcomes and interventions must be disjoint sets.")

    print("✓ Preconditions satisfied")
    print(f"  Graph nodes V: {all_nodes}")
    print(f"  Outcomes Y: {outcomes}")
    print(f"  Interventions W: {interventions}\n")

    # line 3 - compute the ancestral closure H in mutilated graph
    graph_minus_interventions = graph.remove_nodes_from(interventions)
    ancestral_closure = graph_minus_interventions.ancestors_inclusive(outcomes)

    print("  Line 3: Ancestral closure H")
    print(f"    G \\ W has nodes: {set(graph_minus_interventions.nodes())}")
    print(f"    H = {ancestral_closure}\n")

    # line 4 - get consolidated districts of H
    h_subgraph = graph_minus_interventions.subgraph(ancestral_closure)
    consolidated_districts = get_graph_consolidated_districts(h_subgraph)

    print("  Line 4: Consolidated districts")
    print(f"    Found {len(consolidated_districts)} district(s):")
    for i, district in enumerate(sorted(consolidated_districts), 1):
        print(f"      District {i}: {district}")
    print()

    # get the apt order for the full graph
    apt_order_full = get_apt_order(graph)
    print(f"  Apt-order (full graph): {apt_order_full}\n")

    # line 5 - for each district, identify Q[C]
    # Q[C] <- makes a call to IDCD in order to do so
    district_distributions = {}

    for i, district_c in enumerate(sorted(consolidated_districts), 1):
        print(f"{'-' * 50}")
        print(f"Line 5 (District {i}/{len(consolidated_districts)})")
        print(f"  C = {district_c}")

        # Get consolidated district of C in full graph G
        consolidated_district_of_c = get_consolidated_district(graph, district_c)
        print(f"  Cd^G(C) = {consolidated_district_of_c}")

        # Initialize Q[Cd^G(C)] using Proposition 9.8
        # Q[D] = ⊗_{S⊆D} P(S | Pred^G_<(S))
        initial_distribution = initialize_district_distribution(
            graph=graph, district=consolidated_district_of_c, apt_order=apt_order_full
        )
        print(f"  Q[Cd^G(C)] = {initial_distribution}")
        print(f"{'-' * 50}")

        try:
            # Call IDCD with initialized distribution
            result = idcd(
                graph=graph,
                outcomes=district_c,
                district=consolidated_district_of_c,
                distribution=initial_distribution,
            )
            print(f"  ✓ IDCD returned: {result}")

            district_distributions[frozenset(district_c)] = result
            print(f"  ✓ District {district_c} identified successfully\n")

        except Unidentifiable as e:
            # Lines 6-8: if Q[C] = FAIL then return FAIL
            print(f"  ✗ District {district_c} is NOT identifiable")
            print(f"  Reason: {e}\n")
            raise Unidentifiable(
                f"Cannot identify P({outcomes} | do({interventions})). "
                f"District {district_c} failed: {e}"
            ) from e

    # line 10 - compute the tensor product of district distributions
    print(f"{'=' * 50}")
    print("Line 10: Tensor product Q[H] = ⊗ Q[C]")
    print(f"{'=' * 50}\n")

    if len(district_distributions) == 1:
        q_h = next(iter(district_distributions.values()))
        print("  Single district - no product needed.")
    else:
        q_h = Product.safe(district_distributions.values())
        print(f"  Product over {len(district_distributions)} districts.")

    print(f"  Q[H] = {q_h}\n")

    # Line 11: Marginalize to get final result
    marginalize_out = ancestral_closure - outcomes
    print("Line 11: Marginalizing to get P(Y | do(W))")

    if marginalize_out:
        print(f"  Marginalizing out: {marginalize_out}")
        result = q_h.marginalize(marginalize_out)
    else:
        print("  No marginalization needed (H = Y)")
        result = q_h

    print(f"\n{'=' * 50}")
    print(f"✅ SUCCESS: P({outcomes} | do({interventions})) is IDENTIFIABLE.")
    print(f"{'=' * 50}")
    print(f"\nFinal result: {result}\n")

    return result

Now we can make a call to the main cyclic ID algorithm below in order to see if several examples of queries are truly identifiable or 
unidentifiable.

In [ ]:
result = cyclic_id(graph_1, outcomes={Y}, interventions={X})

In [ ]:
result = cyclic_id(graph_1, outcomes={X}, interventions={Y})

In [ ]:
result = cyclic_id(graph_1, outcomes={Y}, interventions={X})

As seen above, the queries for the example minimal graph are all considered identifiable. Below we can create a more complex graph that does have a few identifiable queries for the algorithm to identify. 



In [ ]:
# Define additional variables
A = Variable("A")
B = Variable("B")
C = Variable("C")
D = Variable("D")
E = Variable("E")

# Create complex cyclic graph
complex_graph = NxMixedGraph.from_edges(
    directed=[
        (A, B),  # External input
        (B, B),  # Self-loop
        (B, C),  # First cycle edge 1
        (C, B),  # First cycle edge 2
        (C, D),  # Connection between cycles
        (D, E),  # Second cycle edge 1
        (E, D),  # Second cycle edge 2
    ]
)

In [ ]:
result = cyclic_id(complex_graph, outcomes={E}, interventions={A})

In [ ]:
result = cyclic_id(complex_graph, outcomes={C}, interventions={B})

In [ ]:
result = cyclic_id(complex_graph, outcomes={E}, interventions={D})

In [ ]:
result = cyclic_id(complex_graph, outcomes={E}, interventions={B})

In [ ]:
result = cyclic_id(complex_graph, outcomes={D}, interventions={A})